<h1 align="center">Space-Lunch</h1>
A journey through coding with the Engineering Process

![EDP](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.sciencebuddies.org%2Fblog%2Fgraphics%2F2013-blog-scientific-method-engineering-design-charts-750px.png&f=1&nofb=1)

# Defining our Problem

## Abstract
Look up places to eat near launch sites

## Objective
Create a Python Jupyter Notebook application that will allow a user to select upcoming SpaceX launches and then display places to eat back to the user on a map.

## Requirements
1. Allow user to select between upcoming SpaceX launches.
2. Display places to eat near selected launch back to user on a map.

## Project Steps:
1. [x] Get the list of upcoming launches including date/time and location. Data source: Probably SpaceX API
2. [x] Extract the relevant data (launchpad location, time of launch).
3. [x] Display the information to the user.
4. [x] Allow user to select between launches. (This gives us the launch location)
5. _ Use launch location to get the restaurants. Data Source: Probably Google API
6. _ Parse the restaurant data
7. _ Display the restaurants on a map.

## Research Data:
Most of our research will be on the data sources we intend to use, so we can link to the documentation for that here:

1. Data source for upcoming launches: https://api.spacexdata.com/v4/launches/upcoming
    * Launchpad value comes back as a very ugly ID number (i.e.: 5e9e4501f509094ba4566f84)
2. Data source to convert launchpad IDs to text: 
3. Data source for places to eat nearby: https://developers.google.com/maps/documentation/places/web-service/search-nearby
    * Required Parameter: Location - The point around which to retrieve place information. This must be specified as latitude,longitude.
    * As this requires our location be in lat/lng format, we will need to convert our launch locations to lat/lng. Use this: https://developers.google.com/maps/documentation/geocoding/requests-geocoding?hl=en

# Prototyping
This section of the notebook is designated just for prototyping and testing out pieces of code. Much of our code is "test as you go", testing out each each step in the project as we program it, but this section set aside for all of our messy experimentation while we try to figure out what kind of approach we want to take to building each step.

In [1]:
# Import Dependencies
import requests
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
import datetime
import time

In [2]:
resp = requests.get("https://api.spacexdata.com/v4/launches/upcoming").json()
resp

[{'fairings': {'reused': None,
   'recovery_attempt': None,
   'recovered': None,
   'ships': []},
  'links': {'patch': {'small': 'https://imgur.com/BrW201S.png',
    'large': 'https://imgur.com/573IfGk.png'},
   'reddit': {'campaign': 'https://www.reddit.com/r/spacex/comments/jhu37i/starlink_general_discussion_and_deployment_thread/',
    'launch': None,
    'media': None,
    'recovery': 'https://www.reddit.com/r/spacex/comments/k2ts1q/rspacex_fleet_updates_discussion_thread/'},
   'flickr': {'small': [], 'original': []},
   'presskit': None,
   'webcast': None,
   'youtube_id': None,
   'article': None,
   'wikipedia': 'https://en.wikipedia.org/wiki/Starlink'},
  'static_fire_date_utc': None,
  'static_fire_date_unix': None,
  'net': False,
  'window': None,
  'rocket': '5e9d0d95eda69973a809d1ec',
  'success': None,
  'failures': [],
  'details': None,
  'crew': [],
  'ships': [],
  'capsules': [],
  'payloads': ['61fc02e1e0dc5662b76489b4'],
  'launchpad': '5e9e4501f509094ba4566f84'

In [3]:
date = resp[0]['date_local']
date

'2022-01-31T19:00:00-05:00'

In [4]:
pad = resp[0]['launchpad']
pad

'5e9e4501f509094ba4566f84'

In [5]:
# That launchpad ID is pretty gross, let's convert it to something we can use!
resp = requests.get("https://api.spacexdata.com/v4/launchpads/" + pad).json()
resp

{'images': {'large': ['https://i.imgur.com/9oEMXwa.png']},
 'name': 'CCSFS SLC 40',
 'full_name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
 'locality': 'Cape Canaveral',
 'region': 'Florida',
 'latitude': 28.5618571,
 'longitude': -80.577366,
 'launch_attempts': 80,
 'launch_successes': 78,
 'rockets': ['5e9d0d95eda69973a809d1ec'],
 'timezone': 'America/New_York',
 'launches': ['5eb87cddffd86e000604b32f',
  '5eb87cdeffd86e000604b330',
  '5eb87cdfffd86e000604b331',
  '5eb87ce0ffd86e000604b332',
  '5eb87ce1ffd86e000604b333',
  '5eb87ce2ffd86e000604b335',
  '5eb87ce3ffd86e000604b336',
  '5eb87ce4ffd86e000604b337',
  '5eb87ce4ffd86e000604b338',
  '5eb87ce5ffd86e000604b339',
  '5eb87ce6ffd86e000604b33a',
  '5eb87ce7ffd86e000604b33b',
  '5eb87ce8ffd86e000604b33c',
  '5eb87ceaffd86e000604b33d',
  '5eb87ceaffd86e000604b33e',
  '5eb87cecffd86e000604b33f',
  '5eb87cedffd86e000604b340',
  '5eb87ceeffd86e000604b341',
  '5eb87cefffd86e000604b342',
  '5eb87cf2ffd86e000604b344',


# Parsing SpaceX Data
Now that we are done prototyping, let's build a block of code to output just the date and location for each upcoming SpaceX launch.

In [6]:
# Get list of upcoming launches from SpaceX API
resp = requests.get("https://api.spacexdata.com/v4/launches/upcoming").json()

# Get date/time and location for each launch
launches = [] # Create a list to store launch information in

for launch in resp: # For each "launch" dictionary in the "resp" list of launches:
    
    date = launch['date_local'] # Retrieve the date for that launch
    pad_id = launch['launchpad']# Retrieve the pad ID for that launch
    
    # Convert the pad ID into a pad name using an additional API call
    resp2 = requests.get("https://api.spacexdata.com/v4/launchpads/" + pad_id).json()
    pad = resp2['full_name'] # Pull the full pad name out of the resp2 dictionary
    
    # Put the date and pad name for this launch in a dictionary and add that to the launches list
    launches.append({'date': date, "pad": pad})

# Convert the launches list of dictionaries to a DataFrame
df = pd.DataFrame(launches)

# Display the list of launches to the user
df

,date,pad
0,2022-01-31T19:00:00-05:00,Cape Canaveral Space Force Station Space Launc...
1,2022-02-13T16:00:00-08:00,Vandenberg Space Force Base Space Launch Compl...
2,2022-02-14T19:00:00-05:00,Cape Canaveral Space Force Station Space Launc...
3,2022-02-24T19:00:00-05:00,Cape Canaveral Space Force Station Space Launc...
4,2022-03-30T20:00:00-04:00,Kennedy Space Center Historic Launch Complex 39A
5,2022-03-31T17:00:00-07:00,Vandenberg Space Force Base Space Launch Compl...


# Get User Selection

In [7]:
# Get the user selection
selection = int(input("Select a launch to visit by the index number: "))
selection

Select a launch to visit by the index number: 4


4

In [8]:
# Based on user selection, get the appropriate pad location.
location = df['pad'][selection]

# Display that pad back to the user
location

'Kennedy Space Center Historic Launch Complex 39A'

# Pull and Parse Restaurant Data
After we have finished retrieving the SpaceX launch Data and asking the user what launch they wish to visit, we can move on to retrieving places to eat near that launch site.

In [9]:
# Import our gkey (Do not forget to make your own config.py)
from config import gkey

In [10]:
# Google geocoding API
# Now we can convert the pad name into lat/lng coordinates using the google geocoding API.
resp = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={gkey}").json()
resp

{'results': [{'address_components': [{'long_name': 'Florida',
     'short_name': 'FL',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Brevard County',
     'short_name': 'Brevard County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']}],
   'formatted_address': 'Kennedy Space Center, FL, USA',
   'geometry': {'location': {'lat': 28.6263826, 'lng': -80.6204703},
    'location_type': 'GEOMETRIC_CENTER',
    'viewport': {'northeast': {'lat': 28.6277315802915,
      'lng': -80.61912131970848},
     'southwest': {'lat': 28.6250336197085, 'lng': -80.6218192802915}}},
   'place_id': 'ChIJZer0yrO74IgRbTR4IzfGf1s',
   'plus_code': {'compound_code': 'J9GH+HR Wilson, FL, USA',
    'global_code': '76WXJ9GH+HR'},
   'types': ['establishment', 'museum', 'point_of_interest']}],
 'status': 'OK'}

In [11]:
# Pulling our lat/lng coordinates out of response
lat = resp['results'][0]['geometry']['location']['lat']
lng = resp['results'][0]['geometry']['location']['lng']
print(lat, lng)

28.6263826 -80.6204703


In [12]:
# google maps restaurant API call
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat}%2C{lng}&radius=20000&type=restaurant&key={gkey}"

resp = requests.get(url).json()
food_df = pd.DataFrame(resp['results'])
list(food_df.columns).index('name') # A way to find the index numbers of a column key
food_df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity
0,OPERATIONAL,"{'location': {'lat': 28.6150543, 'lng': -80.82...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Dixie Crossroads,{'open_now': False},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJiYHNYshM54gRLfNszvkj_jo,"{'compound_code': 'J58H+2V Titusville, FL, USA...",2.0,4.5,ChIJiYHNYshM54gRLfNszvkj_jo,GOOGLE,"[restaurant, food, point_of_interest, establis...",7466,"1475 Garden Street, Titusville"
1,OPERATIONAL,"{'location': {'lat': 28.6151324, 'lng': -80.80...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Papa John's Pizza,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJX1V-CTWz4IgRdnwytl-A7As,"{'compound_code': 'J58V+34 Titusville, FL, USA...",1.0,3.5,ChIJX1V-CTWz4IgRdnwytl-A7As,GOOGLE,"[restaurant, food, point_of_interest, establis...",186,"3 South Washington Avenue, Titusville"
2,OPERATIONAL,"{'location': {'lat': 28.5745202, 'lng': -80.81...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,La Cita Golf & Country Club,{'open_now': False},"[{'height': 666, 'html_attributions': ['<a hre...",ChIJUZW7AjNN54gRNmaAi_Bk1e0,"{'compound_code': 'H5FQ+R9 Titusville, FL, USA...",NaN,3.8,ChIJUZW7AjNN54gRNmaAi_Bk1e0,GOOGLE,"[restaurant, food, health, point_of_interest, ...",113,"777 Country Club Drive, Titusville"
3,OPERATIONAL,"{'location': {'lat': 28.558424, 'lng': -80.807...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Gold Lion Bar and deli,{'open_now': True},"[{'height': 3096, 'html_attributions': ['<a hr...",ChIJybaTda2y4IgRwgwXu6liN4s,"{'compound_code': 'H55R+9X Titusville, FL, USA...",1.0,3.9,ChIJybaTda2y4IgRwgwXu6liN4s,GOOGLE,"[restaurant, night_club, bar, food, point_of_i...",379,"4580 Coquina Avenue, Titusville"
4,OPERATIONAL,"{'location': {'lat': 28.5632952, 'lng': -80.79...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,El Leoncito Mexican and Cuban Restaurant,{'open_now': True},"[{'height': 5038, 'html_attributions': ['<a hr...",ChIJaTAcDLqy4IgRZ5W7n697puQ,"{'compound_code': 'H672+8C Titusville, FL, USA...",2.0,4.3,ChIJaTAcDLqy4IgRZ5W7n697puQ,GOOGLE,"[restaurant, food, point_of_interest, establis...",3389,"4280 South Washington Avenue, Titusville"
5,OPERATIONAL,"{'location': {'lat': 28.57697229999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Loyd Have Mercy Restaurant,{'open_now': False},"[{'height': 1153, 'html_attributions': ['<a hr...",ChIJP1RJrM9M54gRuOv1MPPHRQs,"{'compound_code': 'H6G2+Q4 Titusville, FL, USA...",1.0,4.4,ChIJP1RJrM9M54gRuOv1MPPHRQs,GOOGLE,"[restaurant, food, point_of_interest, establis...",1034,"3434 South Washington Avenue, Titusville"
6,OPERATIONAL,"{'location': {'lat': 28.5827467, 'lng': -80.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,{'open_now': False},"[{'height': 940, 'html_attributions': ['<a hre...",ChIJ_--ZMYqw4IgR7udXuupL9Pc,"{'compound_code': 'H8MW+3V Titusville, FL, USA...",1.0,3.5,ChIJ_--ZMYqw4IgR7udXuupL9Pc,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",13,"Kennedy Space Center, Nasa Parkway E Nasa Mff ..."
7,OPERATIONAL,"{'location': {'lat': 28.5244216, 'lng': -80.64...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,{'open_now': False},"[{'height': 940, 'html_attributions': ['<a hre...",ChIJXyQXleiv4IgRFhdBcRfu0TU,{'compound_code': 'G9F2+QG Cape Canaveral Air ...,1.0,3.7,ChIJXyQXleiv4IgRFhdBcRfu0TU,GOOGLE,"[meal_takeaway, restau

In [13]:
restaurants_df = food_df.loc[:,['name','geometry']]
restaurants_df.set_index('name')

,geometry
name,
Dixie Crossroads,"{'location': {'lat': 28.6150543, 'lng': -80.82..."
Papa John's Pizza,"{'location': {'lat': 28.6151324, 'lng': -80.80..."
La Cita Golf & Country Club,"{'location': {'lat': 28.5745202, 'lng': -80.81..."
Gold Lion Bar and deli,"{'location': {'lat': 28.558424, 'lng': -80.807..."
El Leoncito Mexican and Cuban Restaurant,"{'location': {'lat': 28.5632952, 'lng': -80.79..."
Loyd Have Mercy Restaurant,"{'location': {'lat': 28.57697229999999, 'lng':..."
Subway,"{'location': {'lat': 28.5827467, 'lng': -80.65..."
Subway,"{'location': {'lat': 28.5244216, 'lng': -80.64..."
Little Caesars Pizza,"{'location': {'lat': 28.583708, 'lng': -80.806..."


In [14]:
# Building our DataFrame using the columns keys
restaurants_df = food_df[['name','geometry']]
restaurants_df 

,name,geometry
0,Dixie Crossroads,"{'location': {'lat': 28.6150543, 'lng': -80.82..."
1,Papa John's Pizza,"{'location': {'lat': 28.6151324, 'lng': -80.80..."
2,La Cita Golf & Country Club,"{'location': {'lat': 28.5745202, 'lng': -80.81..."
3,Gold Lion Bar and deli,"{'location': {'lat': 28.558424, 'lng': -80.807..."
4,El Leoncito Mexican and Cuban Restaurant,"{'location': {'lat': 28.5632952, 'lng': -80.79..."
5,Loyd Have Mercy Restaurant,"{'location': {'lat': 28.57697229999999, 'lng':..."
6,Subway,"{'location': {'lat': 28.5827467, 'lng': -80.65..."
7,Subway,"{'location': {'lat': 28.5244216, 'lng': -80.64..."
8,Little Caesars Pizza,"{'location': {'lat': 28.583708, 'lng': -80.806..."
9,Subway,"{'location': {'lat': 28.5999079, 'lng': -80.80..."


In [15]:
lats = []
lngs = []
# coords = []
for row in restaurants_df['geometry']:
    lats.append(row['location']['lat'])
    lngs.append(row['location']['lng'])
    
print(lats)
print()
print(lngs)

[28.6150543, 28.6151324, 28.5745202, 28.558424, 28.5632952, 28.57697229999999, 28.5827467, 28.5244216, 28.583708, 28.5999079, 28.4948525, 28.5888465, 28.5833424, 28.57338, 28.5865797, 28.6123602, 28.5858095, 28.58590299999999, 28.612717, 28.58582359999999]

[-80.82032269999999, -80.8071952, -80.8115351, -80.80756199999999, -80.79890379999999, -80.7997324, -80.65281089999999, -80.6486511, -80.80606999999999, -80.80721950000002, -80.5785391, -80.80281529999999, -80.8014084, -80.7980201, -80.8024879, -80.8074347, -80.80277749999999, -80.80342, -80.80919399999999, -80.80477049999999]


In [16]:
restaurants_df['lats'] = lats
restaurants_df.loc[:, 'lngs'] = lngs
restaurants_df

C:\Users\farad\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\farad\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\farad\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,name,geometry,lats,lngs
0,Dixie Crossroads,"{'location': {'lat': 28.6150543, 'lng': -80.82...",28.615054,-80.820323
1,Papa John's Pizza,"{'location': {'lat': 28.6151324, 'lng': -80.80...",28.615132,-80.807195
2,La Cita Golf & Country Club,"{'location': {'lat': 28.5745202, 'lng': -80.81...",28.574520,-80.811535
3,Gold Lion Bar and deli,"{'location': {'lat': 28.558424, 'lng': -80.807...",28.558424,-80.807562
4,El Leoncito Mexican and Cuban Restaurant,"{'location': {'lat': 28.5632952, 'lng': -80.79...",28.563295,-80.798904
5,Loyd Have Mercy Restaurant,"{'location': {'lat': 28.57697229999999, 'lng':...",28.576972,-80.799732
6,Subway,"{'location': {'lat': 28.5827467, 'lng': -80.65...",28.582747,-80.652811
7,Subway,"{'location': {'lat': 28.5244216, 'lng': -80.64...",28.524422,-80.648651
8,Little Caesars Pizza,"{'location': {'lat': 28.583708, 'lng': -80.806...",28.583708,-80.806070
9,Subway,"{'location': {'lat': 28.5999079, 'lng': -80.80...",28.599908,-80.807220


In [17]:
restaurants_df[["lats","lngs"]] # Double "[]" for keying with a list

,lats,lngs
0,28.615054,-80.820323
1,28.615132,-80.807195
2,28.574520,-80.811535
3,28.558424,-80.807562
4,28.563295,-80.798904
5,28.576972,-80.799732
6,28.582747,-80.652811
7,28.524422,-80.648651
8,28.583708,-80.806070
9,28.599908,-80.807220


In [18]:
fig = gmaps.figure()
markers = gmaps.marker_layer(restaurants_df[["lats","lngs"]])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))